In [1]:
import tabula
import pandas as pd
import os
from pathlib import Path
import re

In [2]:
file_name = os.path.expanduser(Path('~/Documents/USICT/syllabus.pdf'))

In [3]:
college = {'college' : 'University School of Information and Communication Technology',
           'courses' : [{
               'course' : 'Bachelor of Technology' ,
               'branches' : [{
                   'branch' : 'Computer Science',
                   'semesters' : [
                       
                   ]
               }]
           }] 
          }

In [4]:
semesters = college['courses'][0]['branches'][0]['semesters'] 

In [5]:
dfs = tabula.read_pdf(file_name,pages='all')

In [6]:
!which python3

/home/parth/Programming/python/get_work_done/env/bin/python3


In [7]:
dfs[2]

,Third Semester,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Code Paper Paper,NaN,L,T/P,C
1,ID,NaN,NaN,NaN,NaN
2,Theory Papers,NaN,NaN,NaN,NaN
3,IT201 15201 Computational Methods,NaN,3,1,4
4,IT203 15203 Circuits and Systems,NaN,3,1,4
5,IT205 15205 Electronic Devices and Circuits,NaN,3,1,4
6,IT207 15207 Object Oriented Programming Using C++,NaN,3,1,4
7,IT209 15209 Computer Graphics,NaN,3,1,4
8,IT211 15211 Database Management Systems,NaN,3,1,4
9,Practical/Viva Voce,NaN,NaN,NaN,NaN


In [8]:
pattern = re.compile(r'(\w+) semester\s*$',re.IGNORECASE)
counter = 1
subject_code_pattern = re.compile(r'([\w ]+)\*? (\d+)\s*$')
code_paper_pattern = re.compile(r'code\s+paper',re.IGNORECASE)
code_paper_paper_pattern = re.compile(r'code\s+paper\s+paper\s*$',re.IGNORECASE)
code_paper_paper_value_pattern = re.compile(r'([\w ]+\*?\s*\d+)\s+([\w\- ]+)')
total_pattern = re.compile(r'\s*total\s*',re.IGNORECASE)

In [9]:
def process_df(df) :
    global counter;
    print(df.columns)
    
    if pattern.match(df.columns[0]) :
        
        sem = {
            'semester' : counter,
            'subjects' : [
                
            ]
        }
        counter += 1
        
        df.columns = df.iloc[0,:]
        df.drop(0,axis=0,inplace=True)
        df.dropna(axis=1,how='all',inplace=True)
        cols = enumerate(temp.columns.isna())
        none_cols = [x[0] for x in cols if x[1]]
        df.drop(df.columns[none_cols],axis=1,inplace=True)
        df.dropna(axis=0,inplace=True)
        df.reset_index(drop = True,inplace=True)
        df.columns = df.columns.str.strip()
        
        temp_df = df.filter(regex=code_paper_paper_pattern,axis=1).copy()
        if not temp_df.empty :
            temp_col = temp_df.iloc[:,0]
            print(temp_col.iloc[-1],total_pattern.match(temp_col.iloc[-1]))
            if total_pattern.match(temp_col.iloc[-1]) :
                temp_col = temp_col.iloc[:-1]
                df.drop(df.tail(1).index,axis=0,inplace = True)
            df['code paper'] = temp_col.apply(lambda x : code_paper_paper_value_pattern.match(x)[1])
            df['Paper'] = temp_col.apply(lambda x : code_paper_paper_value_pattern.match(x)[2])
            df.drop(temp_df.columns,axis=1,inplace=True)
        
        df.rename({'Paper' : 'subject',
                   'Credits' : 'credits',
                   'C' : 'credits'
                  },
                  axis = 1,
                  inplace = True
                 )
        df.rename(lambda x : 'subjectCode' if code_paper_pattern.match(x) else x,
                  axis=1,
                  inplace=True
                 )
        df['subjectCode'] = df['subjectCode'].apply(lambda x : subject_code_pattern.match(x)[1])
        df.drop(['L','T/P'],axis=1,inplace=True)
        sem['subjects'] = df.to_dict('records')
        semesters.append(sem)

In [10]:
SEM = 6
temp = dfs[SEM].copy()
process_df(temp)

Index(['Seventh Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4'],
      dtype='object')
Total <re.Match object; span=(0, 5), match='Total'>


In [11]:
temp

,credits,subjectCode,subject
0,4,IT401,Advanced Computer Networks
1,4,IT403,Software Testing
2,4,IT405,Distributed Systems
3,4,IT407,Artificial Intelligence
4,4,IT409,Simulation and Modeling
5,4,IT411,Digital Image Processing
6,4,IT413,Front End Design Tools and Web Technologies
7,4,IT415,Advanced Java Programming
8,1,IT451,ACN Lab
9,1,IT461,Software Testing Lab


In [12]:
for df in dfs :
    temp = df.copy()
    process_df(temp)

Index(['First Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3'],
      dtype='object')
Index(['Second Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3'],
      dtype='object')
Index(['Third Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3'],
      dtype='object')
Total <re.Match object; span=(0, 5), match='Total'>
Index(['Fourth Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3'],
      dtype='object')
Index(['Fifth Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4'],
      dtype='object')
Index(['Sixth Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3'],
      dtype='object')
Index(['Seventh Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4'],
      dtype='object')
Total <re.Match object; span=(0, 5), match='Total'>
Index(['Eighth Semester', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3'

In [13]:
college

{'college': 'University School of Information and Communication Technology',
 'courses': [{'course': 'Bachelor of Technology',
   'branches': [{'branch': 'Computer Science',
     'semesters': [{'semester': 1,
       'subjects': [{'credits': '4',
         'subjectCode': 'IT401',
         'subject': 'Advanced Computer Networks'},
        {'credits': '4',
         'subjectCode': 'IT403',
         'subject': 'Software Testing'},
        {'credits': '4',
         'subjectCode': 'IT405',
         'subject': 'Distributed Systems'},
        {'credits': '4',
         'subjectCode': 'IT407',
         'subject': 'Artificial Intelligence'},
        {'credits': '4',
         'subjectCode': 'IT409',
         'subject': 'Simulation and Modeling'},
        {'credits': '4',
         'subjectCode': 'IT411',
         'subject': 'Digital Image Processing'},
        {'credits': '4',
         'subjectCode': 'IT413',
         'subject': 'Front End Design Tools and Web Technologies'},
        {'credits': '4',

In [14]:
import requests

In [15]:
query = {
    'college' : 'University School of Information and Communication Technology'
}

In [16]:
r = requests.post('http://localhost:3000/academia/college',json=query)

In [81]:
r = requests.post('http://localhost:3000/academia/college',json=college)

In [82]:
r

<Response [200]>

In [24]:
query = {'college' : 'University School of Information and Communication Technology' , 'course' : 'Bachelor of Technology'}
query['branch'] = college['courses'][0]['branches'][0]

In [25]:
query

{'college': 'University School of Information and Communication Technology',
 'course': 'Bachelor of Technology',
 'branch': {'branch': 'Electronics and Communications Engineering',
  'semesters': [{'semester': 1,
    'subjects': [{'credits': '4',
      'subjectCode': 'IT401',
      'subject': 'Advanced Computer Networks'},
     {'credits': '4', 'subjectCode': 'IT403', 'subject': 'Software Testing'},
     {'credits': '4',
      'subjectCode': 'IT405',
      'subject': 'Distributed Systems'},
     {'credits': '4',
      'subjectCode': 'IT407',
      'subject': 'Artificial Intelligence'},
     {'credits': '4',
      'subjectCode': 'IT409',
      'subject': 'Simulation and Modeling'},
     {'credits': '4',
      'subjectCode': 'IT411',
      'subject': 'Digital Image Processing'},
     {'credits': '4',
      'subjectCode': 'IT413',
      'subject': 'Front End Design Tools and Web Technologies'},
     {'credits': '4',
      'subjectCode': 'IT415',
      'subject': 'Advanced Java Programmin

In [27]:
r = requests.post('http://localhost:3000/academia/branch',json=query)

In [462]:
print(r)

<Response [200]>


In [387]:
q = {'college' : 'University School of Information and Communication Technology'}


In [388]:
r = requests.get('http://localhost:3000/academia/college',json=q)

In [389]:
r.content

b'{"_id":"5f33ab53dc53d8a897ccb5cf","college":"University School of Information and Communication Technology","courses":[{"branches":[{"semesters":[{"subjects":[{"subjectCode":"HS101","_id":"5f33ab53dc53d8a897ccb5d3","subject":"Communication Skills-I","credits":3},{"subjectCode":"BA103","_id":"5f33ab53dc53d8a897ccb5d4","subject":"Chemistry \xe2\x80\x93 I","credits":3},{"subjectCode":"IT 105","_id":"5f33ab53dc53d8a897ccb5d5","subject":"Introduction to Computers","credits":3},{"subjectCode":"IT 107","_id":"5f33ab53dc53d8a897ccb5d6","subject":"Electrical Science","credits":4},{"subjectCode":"BA109","_id":"5f33ab53dc53d8a897ccb5d7","subject":"Mathematics \xe2\x80\x93 I","credits":4},{"subjectCode":"BA111","_id":"5f33ab53dc53d8a897ccb5d8","subject":"Physics \xe2\x80\x93 I","credits":3},{"subjectCode":"HS119","_id":"5f33ab53dc53d8a897ccb5d9","subject":"Impact of Science & Technology on","credits":1},{"subjectCode":"BA151","_id":"5f33ab53dc53d8a897ccb5da","subject":"Chemistry-I Lab","credits"